CBOW Embedding

In [2]:
import torch
import torch.nn as nn

def make_context_vector(context, word_to_ix):
    idxs = [word_to_ix[w] for w in context]
    return torch.tensor(idxs, dtype=torch.long)

In [19]:
CONTEXT_SIZE = 2  # 2 words to the left, 2 to the right
EMDEDDING_DIM = 100

raw_text = """eaders of the global South have long accused Western countries of hypocrisy, and their complaints seem to grow louder by the day. These leaders feel emboldened to challenge Western dominance because they see the world becoming increasingly multipolar. The trend has not been lost on China and Russia, which go to great lengths to fuel resentment against the U.S.-led order.

Charges of hypocrisy against the West are often accurate and fair. Hypocrisy occurs when political leaders conduct foreign policy in ways that are inconsistent with their rhetorical claims of moral virtue, passing over alternative policies that correspond to their stated beliefs. Consider the invasion of Iraq by the United States two decades ago. Washington sold the invasion to the public as virtuous—a way to bolster democracy, human rights, and the rules-based order. The United States could have dealt with the Iraqi leader Saddam Hussein in accordance with the principles of the liberal international order by, for example, securing authorization for the invasion from the UN Security Council. Had Washington pursued other options, it would not have caused so much carnage in Iraq and destabilized the country and the broader region. And the United States would have avoided the charge of hypocrisy that has tainted U.S. foreign policy ever since.

But looked at another way, such accusations of hypocrisy are a testament to the uniqueness of U.S. power. Critics accuse the United States of being hypocritical more than any other country in the West. This is not the result of a flaw in the United States’ character but because of the nature of U.S. power. The United States has built its authority by delivering global public goods through universal institutions. It helps achieve peace and security through the United Nations, free trade through the World Trade Organization, development through the World Bank, and financial assistance through the International Monetary Fund. More than any other country, the United States makes ambitious claims about the general good of its actions. Washington is so often hypocritical because it couches its foreign policy in a language of moral virtue.   

Stay informed.
In-depth analysis delivered weekly.
TWO FACED, TOO FURIOUS
When Western foreign policy is widely perceived as hypocritical, the U.S.-led order becomes costlier to sustain. Policy that is perceived as deceitful undermines the legitimacy of the rules and institutions that underpin it. If the order lacks legitimacy, the United States needs to rely on coercion rather than acceptance, as it can no longer expect the deference of others. Its foreign policy becomes more violent and intolerant of critics, eroding the liberal features that have been prominent in the practice of U.S. power.

Countries that see Western policy as hypocritical may question whether Western officials will act in good faith and in the best interests of their allies. In such circumstances, they may forgo cooperating with the West even when it would have otherwise been beneficial. Take the lukewarm response across the global South to the Biden administration’s Summit for Democracy: U.S. support for autocrats, including those leading Egypt, Saudi Arabia, Singapore, and Vietnam, makes many countries skeptical of a U.S.-led initiative on democracy.

Hypocrisy can also trigger moral outrage. Many people consider hypocrisy to be worse than lying. Whereas liars mislead for gain, hypocrites go a step further by deceiving others while seeking praise for their moral virtue. They feign superiority in the process of violating the very principles they profess to uphold. Non-Western states sometimes respond to hypocrisy by seeking retribution. For example, many signatories to the U.S.-led Treaty on the Non-Proliferation of Nuclear Weapons have criticized the United States for hypocritically calling for nuclear disarmament while actively modernizing its nuclear arsenal. Many of these signatories have reacted to U.S. hypocrisy by adopting a rival accord, the Treaty on the Prohibition of Nuclear Weapons, which aims to make nuclear weapons illegal under international law, thereby challenging the United States’ own possession of nuclear weapons. The signatories of both treaties sought to undermine the United States, even when many of them benefited from protection under Washington’s nuclear umbrella. 

BEWARE OF ANSWERED PRAYERS
It is entirely legitimate and appropriate to denounce Western hypocrisy and its nefarious consequences for people around the globe. But those who accuse the United States and its allies of hypocrisy should also recognize that pointing out hypocrisy can lead to positive change.

The act of trying to deceive others into thinking that Western foreign policies are guided by principles actually strengthens the principles at hand. When governments publicly make excuses for not acting virtuously, they are conceding that those virtues matter. This forces the hypocrites to, on occasion, mend their ways and start conducting foreign affairs in accordance with their purported principles. For example, criticism of Western hypocrisy played a critical role in ending the slave trade, curbing the use of weapons of mass destruction, and cementing the norms of respecting sovereignty and avoiding intervention.

Countries in the global South can shame great powers into changing rules and institutions.
The alternative—a world in which great powers do not even bother justifying their actions on the basis of moral values—would be far more harmful to weaker countries. The pretense of virtue among liberal great powers allows for progress because it grants critics an opportunity to denounce hypocrisy and appeal to higher principles in demanding improvements. Countries in the global South can shame great powers into changing rules and institutions for the better.  

Those in the global South who shout against Western hypocrisy should also beware of the risk of being hypocritical themselves. Many critics tend to denounce the West selectively, criticizing only those instances of Western hypocrisy that hurt their interests directly but keeping quiet whenever it benefits them. India for decades loudly protested Washington’s refusal to lead a global process to rid the world of nuclear weapons, only to toe the line the minute it secured concessions and signed a civil nuclear agreement with the United States in 2005.

Finally, countries in the global South should recognize that too much criticism of hypocrisy can endanger international cooperation by breeding cynicism and political paralysis. Hypocrisy can sometimes be useful. It provides governments a pragmatic way out in situations when valuable principles are in conflict. Take the case of the Inflation Reduction Act introduced by the Biden administration. The law provides subsidies for industries to transition to low carbon energy sources and thereby reflects a commitment to mitigating climate change for the entire planet. But the IRA also violates the norms of free trade that the United States so forcefully applies to others. Hypocrisy in this case allows the White House to proclaim the value of both protecting the planet and maintaining free trade, even if the administration is not able to reconcile the two.

A LITTLE HYPOCRISY, WELL DONE
Western hypocrisy can be beneficial, as long as it is handled well. This requires policymakers in the Western alliance to get their response right whenever they are confronted with their failure to live up to their moral commitments. Rather than merely reaffirming the value of the principle that they are violating, they should specify measures to comply with it. Such a proactive response has the advantage of showing the world that in the face of criticism, the Western international order is capable of learning, adapting, and evolving.

By responding to charges of hypocrisy by doing better in the future, the United States and its allies can prepare for a more competitive and conflictual world. Countries of the global South have rarely accused Beijing of hypocrisy, in part because China has shied away from articulating a coherent vision of international order. But as the country grows more powerful and influential, its policymakers will be forced to present to the world ideas and projects that will require some kind of appeal to virtue and principle. In turn, this will inevitably result in the details of Chinese foreign policy grating against some of the country’s professed values. As its clout in world politics expands, Beijing will increasingly face complaints of hypocrisy. And when that day comes, people the world over may find that hypocritical behavior under the banner of liberal values was not that bad after all.""".split()

# By deriving a set from `raw_text`, we deduplicate the array
vocab = set(raw_text)
vocab_size = len(vocab)

word_to_ix = {word: i for i, word in enumerate(vocab)}
ix_to_word = {ix:word for ix, word in enumerate(vocab)}

data = []
for i in range(CONTEXT_SIZE, len(raw_text) - CONTEXT_SIZE):
    context = (
        [raw_text[i - j - 1] for j in range(CONTEXT_SIZE)]
        + [raw_text[i + j + 1] for j in range(CONTEXT_SIZE)]
    )
    target = raw_text[i]
    data.append((context, target))

In [48]:
class CBOW(nn.Module):

    def __init__(self, vocab_size, embedding_dim):
        super(CBOW, self).__init__()
        self.embedding = nn.Embedding(vocab_size,embedding_dim)
        self.linear1 = nn.Linear(embedding_dim, 128)
        self.activate1 = nn.ReLU()

        self.linear2 = nn.Linear(128, vocab_size)
        self.activate2 = nn.LogSoftmax(dim=-1)

    def forward(self, inputs):
        embeds = sum(self.embedding(inputs)).view(1,-1)
        out = self.linear1(embeds)
        out = self.activate1(out)
        out = self.linear2(out)
        out = self.activate2(out)
        return out
    
    def get_word_embedding(self, word):
        word_idx = torch.tensor([word_to_ix(word)])
        return self.embedding(word_idx).view(1,-1)


In [49]:
model = CBOW(vocab_size, EMDEDDING_DIM)

loss_function = nn.NLLLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001)

In [ ]:
for epoch in range(500):
    total_loss = 0

    for context, target in data:
        context_vector = make_context_vector(context, word_to_ix)  

        log_probs = model(context_vector)

        total_loss += loss_function(log_probs, torch.tensor([word_to_ix[target]]))
    print("Epoc:", epoch, ", total loss:", total_loss)
    #optimize at the end of each epoch
    optimizer.zero_grad()
    total_loss.backward()
    optimizer.step()

In [ ]:
#TESTING
context = ['rarely','accused','of', 'hypocrisy']
context_vector = make_context_vector(context, word_to_ix)
a = model(context_vector)

In [ ]:
#Print result
print(f'Raw text: {" ".join(raw_text)}\n')
print(f'Context: {context}\n')
print(f'Prediction: {ix_to_word[torch.argmax(a[0]).item()]}')